In [ ]:
import requests
import urllib3
import time
from datetime import datetime, timedelta

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# ===== API Configuration =====
TOKEN_URL = 'https://192.168.100.81:32222/auth/realms/icedq/protocol/openid-connect/token'
token_data = {
    'grant_type': 'password',
    'client_id': 'icedq.admin-ui',
    'client_secret': 'b2e25665-7fce-49fa-9c63-f7c872dabcf2',
    'username': 'tapshyu',
    'password': 'Admin@1234'
}
# ===== Token Management =====
def get_access_token():
    """Fetch a new token and record its expiry time."""
    resp = requests.post(TOKEN_URL, data=token_data, verify=False)
    if resp.status_code != 200:
        raise Exception(f"❌ Failed to get token: {resp.text}")

    data = resp.json()
    token = data.get("access_token")
    expires_in = data.get("expires_in", 300)  # default 5 mins if not provided
    if not token:
        raise Exception("❌ Token not found in response.")

    expiry_time = datetime.now() + timedelta(seconds=expires_in - 30)  # refresh 30 sec before expiry
    return token, expiry_time
# =======Store Rule Ids================
Rule_Ids =['rule-d1244036-fd60-5ea5-bd44-649ce2b8d1e6',
           'rule-9217ef3b-1e60-5fa1-8f60-9c91f2c84f29',
           'rule-490ba93d-ef3a-5f52-824d-cefa57407434',
           'rule-6ae3418d-1ef5-52db-b76e-cdcbff8747d1',
           'rule-ae598ae5-28e8-560d-83ab-14f7b8403d0a']
common_headers = {
    'accept': 'application/json, text/plain, */*',
    'content-type': 'application/json',
    'org-id': 'org-icedq',
    'workspace-id': 'wksc-946e5c43-2b8f-54a8-b213-1e899d435aca'
}
# Initial token
access_token, token_expiry = get_access_token()
headers = {**common_headers, 'authorization': f"Bearer {access_token}"}

#----URL's----

get_Rule_URL = 'https://192.168.100.81:32222/api/v1/rules/[rule_id]'
checkout_rule_URL = 'https://192.168.100.81:32222/api/v1/rules/{{rule_id}}/checkout'
stage_rule_URL = 'https://192.168.100.81:32222/api/v1/rules/{{rule_id}}'
run_the_rule_forDebug = 'https://192.168.100.81:32222/api/v1/workflow:trigger'
run_the_rule_payload_forDebug = {"objectId": "{{rule_id}}"}
discard_the_checkout_rule = 'https://192.168.100.81:32222/api/v1/rules/{{rule_id}}:discard'
run_the_rule = 'https://192.168.100.81:32222/api/v1/workflow:trigger'
run_the_rule_payload = {"objectId": "{{rule_id}}"}





In [2]:
import time
import requests
import urllib3
from datetime import datetime, timedelta

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


# ================= Configuration =================
TOKEN_URL = 'https://192.168.100.81:32222/auth/realms/icedq/protocol/openid-connect/token'
RULE_IDS = [
    'rule-9217ef3b-1e60-5fa1-8f60-9c91f2c84f29',
    'rule-490ba93d-ef3a-5f52-824d-cefa57407434',
    'rule-6ae3418d-1ef5-52db-b76e-cdcbff8747d1',
    'rule-ae598ae5-28e8-560d-83ab-14f7b8403d0a',
    'rule-4f23edd4-9d59-5529-8c36-a5ce463400bf'
]

token_data = {
    'grant_type': 'password',
    'client_id': 'icedq.admin-ui',
    'client_secret': 'b2e25665-7fce-49fa-9c63-f7c872dabcf2',
    'username': 'tapshyu',
    'password': 'Admin@1234'
}

COMMON_HEADERS = {
    'accept': 'application/json, text/plain, */*',
    'content-type': 'application/json',
    'org-id': 'org-icedq',
    'workspace-id': 'wksc-946e5c43-2b8f-54a8-b213-1e899d435aca'
}

API_ENDPOINTS = {
    "get": "https://192.168.100.81:32222/api/v1/rules/{rule_id}",
    "checkout": "https://192.168.100.81:32222/api/v1/rules/{rule_id}/checkout",
    "stage": "https://192.168.100.81:32222/api/v1/internal/rules/{rule_id}",
    "validate":"https://192.168.100.81:32222/api/v1/internal/rules/{rule_id}:validate",
    "stage_rule":"https://192.168.100.81:32222/api/v1/rules/{rule_id}:stage",
    "run": "https://192.168.100.81:32222/api/v1/workflow:trigger",
    "discard": "https://192.168.100.81:32222/api/v1/rules/{rule_id}:discard"
}


# ================= Token Manager =================
class TokenManager:
    def __init__(self):
        self.token = None
        self.expiry_time = datetime.now()

    def get_token(self) -> str:
        """Return a valid token, refreshing if needed."""
        if not self.token or datetime.now() >= self.expiry_time:
            resp = requests.post(TOKEN_URL, data=token_data, verify=False)
            resp.raise_for_status()
            data = resp.json()

            self.token = data["access_token"]
            expires_in = data.get("expires_in", 300)
            self.expiry_time = datetime.now() + timedelta(seconds=expires_in - 30)

        return self.token


# ================= Rule Manager =================
class RuleManager:
    def __init__(self, token_manager: TokenManager):
        self.session = requests.Session()
        self.token_manager = token_manager

    def _make_request(self, method: str, url: str, payload=None):
        headers = {**COMMON_HEADERS, "authorization": f"Bearer {self.token_manager.get_token()}"}
        resp = self.session.request(method, url, headers=headers, json=payload, verify=False)

        print(f"\n➡️ {method.upper()} {url}")
        if payload:
            print(f"   📤 Payload: {payload}")

        if resp.ok:
            try:
                data = resp.json()
                print(f"   📥 Response: {data}")
                return data
            except ValueError:
                print(f"   📥 Response: {resp.text}")
                return resp.text
        else:
            print(f"❌ {resp.status_code}: {resp.text}")
            return None

    def process_rule(self, rule_id: str):
        print(f"\n================= 🔹 Processing rule: {rule_id} =================")

        # Replace rule_id in endpoints
        get_url = API_ENDPOINTS["get"].format(rule_id=rule_id)
        checkout_url = API_ENDPOINTS["checkout"].format(rule_id=rule_id)
        stage_url = API_ENDPOINTS["stage"].format(rule_id=rule_id)
        discard_url = API_ENDPOINTS["discard"].format(rule_id=rule_id)
        Validateurl = API_ENDPOINTS["validate"].format(rule_id=rule_id)
        stage_url_afteredit = API_ENDPOINTS["stage_rule"].format(rule_id=rule_id)

        # 1. GET rule → modify payload
        rule_data = self._make_request("GET", get_url)

        # If rule_data contains "name", append "_v1"
        if isinstance(rule_data, dict) and "name" in rule_data:
            original_name = rule_data["name"]
            rule_data["name"] = f"{original_name}_v1"
            print(f"   ✏️ Updated name: {original_name} → {rule_data['name']}")

        # 2. Checkout rule
        self._make_request("POST", checkout_url)

        # 3. Stage rule (payload = updated rule_data)
        if rule_data:
            self._make_request("PUT", stage_url, payload=rule_data)

        # 4. Validate rule
        self._make_request("POST", Validateurl, payload={})       

        # 5. Stage rule after validate
        self._make_request("POST", stage_url_afteredit, payload={})   

        # 6. Run rule
        self._make_request("POST", API_ENDPOINTS["run"], payload={"isDebugRun":"true","objectId": rule_id})

        # 7. Discard rule
        self._make_request("POST", discard_url)

        # 8. Run again after discard
        self._make_request("POST", API_ENDPOINTS["run"], payload={"objectId": rule_id})


# ================= Main Script =================
if __name__ == "__main__":

    token_manager = TokenManager()
    rule_manager = RuleManager(token_manager)

    for rid in RULE_IDS:
        rule_manager.process_rule(rid)
        time.sleep(1)  # avoid flooding server



================= 🔹 Processing rule: rule-9217ef3b-1e60-5fa1-8f60-9c91f2c84f29 =================

➡️ GET https://192.168.100.81:32222/api/v1/rules/rule-9217ef3b-1e60-5fa1-8f60-9c91f2c84f29
   📥 Response: {'id': 'rule-9217ef3b-1e60-5fa1-8f60-9c91f2c84f29', 'orgId': 'org-icedq', 'accountId': 'acct-65584c43-c749-5474-86cf-a50934d7c1a0', 'workspaceId': 'wksc-946e5c43-2b8f-54a8-b213-1e899d435aca', 'folderId': 'fldr-1b782cff-6fd1-55f2-b6f3-5fbcc9bd9345', 'template': 'recon', 'ruleType': 'Recon', 'name': 'Rule_Azure_Analysisi_Service_Both_Authentication', 'state': 'Published', 'code': 'Rule_Azure_Analysisi_Service_Both_Authentication', 'description': 'Rule_Azure_Analysisi_Service_Both_Authentication', 'purpose': 'Rule_Azure_Analysisi_Service_Both_Authentication', 'createdBy': 'tapshyu', 'updatedBy': 'tapshyu', 'createdTimestamp': '09/09/2025 09:26:18 UTC', 'updatedTimestamp': '09/22/2025 09:02:47 UTC', 'isActive': True, 'isDeleted': False, 'properties': {'criticality': 'Warning', 'engineType

In [ ]:
import time
import requests
import urllib3
from datetime import datetime, timedelta

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


# ================= Configuration =================
TOKEN_URL = 'https://192.168.100.81:32222/auth/realms/icedq/protocol/openid-connect/token'
RULE_IDS = [
    'rule-66ce6bec-7598-54f6-a89d-f8efbf34298f',
    'rule-64aca6b5-cb00-5800-9241-5598bac2e714',
    'rule-e0d3ed98-9826-570c-b0b3-55232bbcbc90',
    'rule-1eb59b84-24a3-54a3-873a-15a831ff0335',
    'rule-8def6781-2c37-5a2c-920c-10a0228541af'
]

token_data = {
    'grant_type': 'password',
    'client_id': 'icedq.admin-ui',
    'client_secret': 'b2e25665-7fce-49fa-9c63-f7c872dabcf2',
    'username': 'tapshyu',
    'password': 'Admin@1234'
}

COMMON_HEADERS = {
    'accept': 'application/json, text/plain, */*',
    'content-type': 'application/json',
    'org-id': 'org-icedq',
    'workspace-id': 'wksc-946e5c43-2b8f-54a8-b213-1e899d435aca'
}

API_ENDPOINTS = {
    "get": "https://192.168.100.81:32222/api/v1/rules/{rule_id}",
    "checkout": "https://192.168.100.81:32222/api/v1/rules/{rule_id}/checkout",
    "stage": "https://192.168.100.81:32222/api/v1/internal/rules/{rule_id}",
    "validate":"https://192.168.100.81:32222/api/v1/internal/rules/{rule_id}:validate",
    "stage_rule":"https://192.168.100.81:32222/api/v1/rules/{rule_id}:stage",
    "run": "https://192.168.100.81:32222/api/v1/workflow:trigger",
    "discard": "https://192.168.100.81:32222/api/v1/rules/{rule_id}:discard"
}


# ================= Token Manager =================
class TokenManager:
    def __init__(self):
        self.token = None
        self.expiry_time = datetime.now()

    def get_token(self) -> str:
        """Return a valid token, refreshing if needed."""
        if not self.token or datetime.now() >= self.expiry_time:
            resp = requests.post(TOKEN_URL, data=token_data, verify=False)
            resp.raise_for_status()
            data = resp.json()

            self.token = data["access_token"]
            expires_in = data.get("expires_in", 300)
            self.expiry_time = datetime.now() + timedelta(seconds=expires_in - 30)

        return self.token


# ================= Rule Manager =================
class RuleManager:
    def __init__(self, token_manager: TokenManager):
        self.session = requests.Session()
        self.token_manager = token_manager

    def _make_request(self, method: str, url: str, payload=None):
        headers = {**COMMON_HEADERS, "authorization": f"Bearer {self.token_manager.get_token()}"}
        resp = self.session.request(method, url, headers=headers, json=payload, verify=False)

        print(f"\n➡️ {method.upper()} {url}")
        if payload:
            print(f"   📤 Payload: {payload}")

        if resp.ok:
            try:
                data = resp.json()
                print(f"   📥 Response: {data}")
                return data
            except ValueError:
                print(f"   📥 Response: {resp.text}")
                return resp.text
        else:
            print(f"❌ {resp.status_code}: {resp.text}")
            return None

    def process_rule(self, rule_id: str):
        print(f"\n================= 🔹 Processing rule: {rule_id} =================")

        # Replace rule_id in endpoints
        get_url = API_ENDPOINTS["get"].format(rule_id=rule_id)
        checkout_url = API_ENDPOINTS["checkout"].format(rule_id=rule_id)
        stage_url = API_ENDPOINTS["stage"].format(rule_id=rule_id)
        discard_url = API_ENDPOINTS["discard"].format(rule_id=rule_id)
        Validateurl = API_ENDPOINTS["validate"].format(rule_id=rule_id)
        stage_url_afteredit = API_ENDPOINTS["stage_rule"].format(rule_id=rule_id)

        # 1. GET rule → modify payload
        rule_data = self._make_request("GET", get_url)

        # If rule_data contains "name", append "_v1"
        if isinstance(rule_data, dict) and "name" in rule_data:
            original_name = rule_data["name"]
            rule_data["name"] = f"{original_name}_v1"
            print(f"   ✏️ Updated name: {original_name} → {rule_data['name']}")

        # 2. Checkout rule
        self._make_request("POST", checkout_url)

        # 3. Stage rule (payload = updated rule_data)
        if rule_data:
            self._make_request("PUT", stage_url, payload=rule_data)

        # 4. Validate rule
        self._make_request("POST", Validateurl, payload={})       

        # 5. Stage rule after validate
        self._make_request("POST", stage_url_afteredit, payload={})   

        # 6. Run rule
        self._make_request("POST", API_ENDPOINTS["run"], payload={"isDebugRun":"true","objectId": rule_id})

        # 7. Discard rule
        self._make_request("POST", discard_url)

        # 8. Run again after discard
        self._make_request("POST", API_ENDPOINTS["run"], payload={"objectId": rule_id})


# ================= Main Script =================
if __name__ == "__main__":

    token_manager = TokenManager()
    rule_manager = RuleManager(token_manager)

    for rid in RULE_IDS:
        rule_manager.process_rule(rid)
        time.sleep(1)  # avoid flooding server



================= 🔹 Processing rule: rule-66ce6bec-7598-54f6-a89d-f8efbf34298f =================

➡️ GET https://192.168.100.81:32222/api/v1/rules/rule-66ce6bec-7598-54f6-a89d-f8efbf34298f
   📥 Response: {'id': 'rule-66ce6bec-7598-54f6-a89d-f8efbf34298f', 'orgId': 'org-icedq', 'accountId': 'acct-65584c43-c749-5474-86cf-a50934d7c1a0', 'workspaceId': 'wksc-946e5c43-2b8f-54a8-b213-1e899d435aca', 'folderId': 'fldr-c476c806-27ce-5d05-a996-dfe7da8159c0', 'template': 'pushdown', 'ruleType': 'Pushdown', 'name': 'Rule with saas', 'state': 'Published', 'createdBy': 'tapshyu', 'updatedBy': 'tapshyu', 'createdTimestamp': '09/24/2025 10:50:16 UTC', 'updatedTimestamp': '09/24/2025 10:50:20 UTC', 'isActive': True, 'isDeleted': False, 'properties': {'criticality': 'Warning', 'engineType': 'HighThroughput', 'stopAtFailAndErrorCount': 0, 'stopAtRowCount': 0, 'stopAtTime': 0, 'degreeOfParallelism': 1, 'engineInfo': {}, 'languageType': 'groovy'}, 'objects': [{'id': 'SourceDataset', 'type': 'source', 'nam